In [1]:
from src import csv_handlers, data_retrieval, string_handlers

In [14]:
data_retrieval.download_bulk_in_batches("2014-01-01", "2018-01-01", "data", award_types=["contracts"])

2019-11-16 11:05:26,183 - src.data_retrieval - DEBUG - Number of batches: 51


  0%|          | 0/51 [00:00<?, ?it/s]

2019-11-16 11:05:30,791 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100530532547.zip`.
2019-11-16 11:05:32,041 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100531560273.zip`.
2019-11-16 11:05:32,950 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100532754774.zip`.
2019-11-16 11:05:33,850 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100533665792.zip`.
2019-11-16 11:05:34,728 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100534542376.zip`.
2019-11-16 11:05:35,612 - src.data_retrieval - DEBUG - Generated file will be at `https://

2019-11-16 11:06:14,396 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100614225826.zip`.
2019-11-16 11:06:15,355 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100615092355.zip`.
2019-11-16 11:06:16,320 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100616125677.zip`.
2019-11-16 11:06:17,210 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100617035798.zip`.
2019-11-16 11:06:18,091 - src.data_retrieval - DEBUG - Generated file will be at `https://files.usaspending.gov/generated_downloads/all_prime_transactions_20191116100617911785.zip`.
2019-11-16 11:12:34,709 - src.data_retrieval - DEBUG - `all_prime_transactions_20191116100

  2%|▏         | 1/51 [07:32<6:16:54, 452.29s/it]

...

2019-11-16 12:43:00,316 - src.data_retrieval - DEBUG - `all_prime_transactions_20191116100555428528.zip` is ready, starting download.
2019-11-16 12:47:31,160 - src.data_retrieval - DEBUG - Download finished, moving to the next.


 98%|█████████▊| 50/51 [1:42:04<05:49, 349.17s/it]

2019-11-16 12:47:32,886 - src.data_retrieval - DEBUG - `all_prime_transactions_20191116100607392659.zip` is ready, starting download.
2019-11-16 12:52:43,237 - src.data_retrieval - DEBUG - Download finished, moving to the next.


100%|██████████| 51/51 [1:47:17<00:00, 338.04s/it]

2019-11-16 12:52:43,238 - src.data_retrieval - DEBUG - All downloads are completed.


In [2]:
csv_handlers.extract_all_recipients("data")

2019-11-16 15:12:22,488 - src.csv_handlers - DEBUG - Found .zip file: data/all_prime_transactions_20191116100608305239.zip
2019-11-16 15:12:22,490 - src.csv_handlers - DEBUG - Found .csv file in zip, opening: all_contracts_prime_transactions_1.csv
2019-11-16 15:12:22,491 - src.csv_handlers - DEBUG - Reading file: <zipfile.ZipExtFile name='all_contracts_prime_transactions_1.csv' mode='r' compress_type=deflate>


KeyboardInterrupt: 

In [2]:
df = csv_handlers.get_recipient_data_from_csv("data/all_recipients.csv")

2019-11-16 15:17:12,320 - src.csv_handlers - DEBUG - Reading file: data/all_recipients.csv


In [3]:
df.head() # 897974 rows

,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_name,recipient_parent_duns,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number
0,3197803,L C IND INCORPORATED (6307),NaN,LC INDUSTRIES INC.,3197803.0,USA,UNITED STATES,4500 EMPEROR BLVD,NaN,DURHAM,NC,NORTH CAROLINA,277038420,4.0,9195968277,9199415195
1,134345169,"HERNDON PRODUCTS, LLC",NaN,HERNDON PRODUCTS LLC,80097712.0,USA,UNITED STATES,3801 LLOYD KING DR,NaN,O FALLON,MO,MISSOURI,633682224,2.0,3147397400,3147395344
2,60605883,"ROCKWELL COLLINS, INC.",NaN,ROCKWELL COLLINS INC.,962960589.0,USA,UNITED STATES,400 COLLINS RD NE,NaN,CEDAR RAPIDS,IA,IOWA,524980505,1.0,3192655467,3192951787
3,877936518,DELL MARKETING L.P.,NaN,DENALI HOLDING INC,601839660.0,USA,UNITED STATES,ONE DELL WAY,NaN,ROUND ROCK,TX,TEXAS,786820001,31.0,5127230669,5122834810
4,809919038,"STAG ENTERPRISE, INC.",NaN,STAG ENTERPRISE INC.,809919038.0,USA,UNITED STATES,383 WILBANKS DR,NaN,BALL GROUND,GA,GEORGIA,301076002,11.0,7707208888,7707208887


In [4]:
df.isnull().sum()

recipient_duns                           2
recipient_name                         108
recipient_doing_business_as_name    859307
recipient_parent_name                 7260
recipient_parent_duns                 4153
recipient_country_code                 175
recipient_country_name              266360
recipient_address_line_1               484
recipient_address_line_2            889735
recipient_city_name                    328
recipient_state_code                 43951
recipient_state_name                263572
recipient_zip_4_code                  8361
recipient_congressional_district    275751
recipient_phone_number              115839
recipient_fax_number                201552
dtype: int64

In [5]:
df['recipient_name'] = df['recipient_name'].combine_first(df['recipient_parent_name'])
df['recipient_parent_name'] = df['recipient_parent_name'].combine_first(df['recipient_name'])
# after the combination of recipient and recipient_parent names drop the still null rows
df = df[~ df.recipient_name.isnull()]

In [6]:
df.isnull().sum()

recipient_duns                           1
recipient_name                           0
recipient_doing_business_as_name    859291
recipient_parent_name                    0
recipient_parent_duns                 4149
recipient_country_code                 174
recipient_country_name              266344
recipient_address_line_1               476
recipient_address_line_2            889719
recipient_city_name                    321
recipient_state_code                 43935
recipient_state_name                263570
recipient_zip_4_code                  8353
recipient_congressional_district    275735
recipient_phone_number              115823
recipient_fax_number                201536
dtype: int64

In [7]:
len(df.recipient_name.unique())

258855

In [8]:
len(df.recipient_parent_name.unique())

277775

In [9]:
df.recipient_name = string_handlers.get_cleaned_series(df.recipient_name)
df.recipient_parent_name = string_handlers.get_cleaned_series(df.recipient_parent_name)

In [10]:
len(df.recipient_name.unique())

250886

In [11]:
len(df.recipient_parent_name.unique())

230728

In [12]:
# only leaves strings and some space
df.recipient_name = string_handlers.get_better_cleaned_series(df.recipient_name)
df.recipient_parent_name = string_handlers.get_better_cleaned_series(df.recipient_parent_name)

In [13]:
len(df.recipient_name.unique())

249600

In [14]:
len(df.recipient_parent_name.unique())

230242

In [15]:
df.head()

,recipient_duns,recipient_name,recipient_doing_business_as_name,recipient_parent_name,recipient_parent_duns,recipient_country_code,recipient_country_name,recipient_address_line_1,recipient_address_line_2,recipient_city_name,recipient_state_code,recipient_state_name,recipient_zip_4_code,recipient_congressional_district,recipient_phone_number,recipient_fax_number
0,3197803,L C IND INCORPORATED,NaN,LC INDUSTRIES INC,3197803.0,USA,UNITED STATES,4500 EMPEROR BLVD,NaN,DURHAM,NC,NORTH CAROLINA,277038420,4.0,9195968277,9199415195
1,134345169,HERNDON PRODUCTS LLC,NaN,HERNDON PRODUCTS LLC,80097712.0,USA,UNITED STATES,3801 LLOYD KING DR,NaN,O FALLON,MO,MISSOURI,633682224,2.0,3147397400,3147395344
2,60605883,ROCKWELL COLLINS INC,NaN,ROCKWELL COLLINS INC,962960589.0,USA,UNITED STATES,400 COLLINS RD NE,NaN,CEDAR RAPIDS,IA,IOWA,524980505,1.0,3192655467,3192951787
3,877936518,DELL MARKETING LP,NaN,DENALI HOLDING INC,601839660.0,USA,UNITED STATES,ONE DELL WAY,NaN,ROUND ROCK,TX,TEXAS,786820001,31.0,5127230669,5122834810
4,809919038,STAG ENTERPRISE INC,NaN,STAG ENTERPRISE INC,809919038.0,USA,UNITED STATES,383 WILBANKS DR,NaN,BALL GROUND,GA,GEORGIA,301076002,11.0,7707208888,7707208887


In [18]:
parent_counts = df.groupby(['recipient_name'])['recipient_parent_name'].nunique()

In [19]:
parent_counts[parent_counts >= 5]

recipient_name
ARMY AIR FORCE EXCHANGE SERVICE                                       5
ARMY UNITED STATES DEPARTMENT OF THE                                  6
CANBERRA INDUSTRIES INC                                               6
CDW GOVERNMENT LLC                                                    5
DELL MARKETING LIMITED PARTNERSHIP                                    5
DELL MARKETING LP                                                     6
DELL SERVICES FEDERAL GOVERNMENT INC                                  5
DEPUY SYNTHES SALES INC                                               6
DYNCORP                                                               6
DYNCORP INTERNATIONAL LIMITED LIABILITY COMPANY                       5
DYNCORP INTERNATIONAL LLC                                             6
EATON CORPORATION                                                     6
EGG TECHNICAL SERVICES INC                                            5
FEDERAL PRISON INDUSTRIES INC                    

In [20]:
df[df.recipient_name == 'DYNCORP'].recipient_parent_name.unique()


array(['DYNCORP INTERNATIONAL LIMITED LIABILITY COMPANY', 'DYNCORP',
       'DYNCORP INTERNATIONAL LLC', 'COMPUTER SCIENCES CORPORATION',
       'VERITAS CAPITAL FUND II LP THE', 'CERBERUS CAPITAL MANAGEMENT LP'],
      dtype=object)

# Some Analysis on the Data - Frequent words/companies

In [22]:
from nltk.probability import FreqDist

In [23]:
FreqDist(df.recipient_name)

FreqDist({'LOCKHEED MARTIN CORPORATION': 1786, 'RAYTHEON COMPANY': 1090, 'HONEYWELL INTERNATIONAL INC': 829, 'NORTHROP GRUMMAN SYSTEMS CORPORATION': 823, 'L COMMUNICATIONS CORPORATION': 822, 'BOEING COMPANY THE': 720, 'SCIENCE APPLICATIONS INTERNATIONAL CORPORATION': 609, 'URS GROUP INC': 564, 'HANGER PROSTHETICS ORTHOTICS INC': 548, 'COMPUTER SCIENCES CORPORATION': 383, ...})

In [24]:
FreqDist(df.recipient_parent_name)

FreqDist({'LOCKHEED MARTIN CORPORATION': 2648, 'GENERAL DYNAMICS CORPORATION': 1798, 'RAYTHEON COMPANY': 1752, 'L COMMUNICATIONS HOLDINGS INC': 1709, 'NORTHROP GRUMMAN CORPORATION': 1620, 'THE BOEING COMPANY': 1553, 'GOVERNMENT OF THE UNITED STATES': 1436, 'BAE SYSTEMS PLC': 1379, 'HONEYWELL INTERNATIONAL INC': 1355, 'GENERAL ELECTRIC COMPANY': 1156, ...})

In [42]:
child_tokens = df.recipient_name.str.split()
child_tokens = [item for sublist in child_tokens for item in sublist]
parent_tokens = df.recipient_parent_name.str.split()
parent_tokens = [item for sublist in parent_tokens for item in sublist]

In [44]:
FreqDist(child_tokens)

FreqDist({'INC': 387528, 'LLC': 144472, 'CORPORATION': 51007, 'COMPANY': 46065, 'SERVICES': 45797, 'OF': 39979, 'INCORPORATED': 37397, 'SYSTEMS': 33821, 'AND': 27429, 'GROUP': 23727, ...})

In [45]:
FreqDist(parent_tokens)

FreqDist({'INC': 398657, 'LLC': 130527, 'CORPORATION': 62586, 'OF': 40861, 'COMPANY': 40409, 'SERVICES': 38148, 'GROUP': 28211, 'SYSTEMS': 27116, 'CO': 26405, 'THE': 26067, ...})

In [46]:
FreqDist(child_tokens + parent_tokens)

FreqDist({'INC': 786185, 'LLC': 274999, 'CORPORATION': 113593, 'COMPANY': 86474, 'SERVICES': 83945, 'OF': 80840, 'SYSTEMS': 60937, 'INCORPORATED': 53513, 'GROUP': 51938, 'CO': 49911, ...})